# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db',engine)
X = df.message
y = df[df.columns[4:]]

In [16]:
y = y.astype(int)

In [17]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26180 entries, 0 to 26179
Data columns (total 36 columns):
related                   26180 non-null int64
request                   26180 non-null int64
offer                     26180 non-null int64
aid_related               26180 non-null int64
medical_help              26180 non-null int64
medical_products          26180 non-null int64
search_and_rescue         26180 non-null int64
security                  26180 non-null int64
military                  26180 non-null int64
child_alone               26180 non-null int64
water                     26180 non-null int64
food                      26180 non-null int64
shelter                   26180 non-null int64
clothing                  26180 non-null int64
money                     26180 non-null int64
missing_people            26180 non-null int64
refugees                  26180 non-null int64
death                     26180 non-null int64
other_aid                 26180 non-null int6

In [7]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2.0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.0,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df['genre'].value_counts()

news      13039
direct    10747
social     2394
Name: genre, dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26180 entries, 0 to 26179
Data columns (total 40 columns):
id                        26180 non-null float64
message                   26180 non-null object
original                  26180 non-null object
genre                     26180 non-null object
related                   26180 non-null float64
request                   26180 non-null float64
offer                     26180 non-null float64
aid_related               26180 non-null float64
medical_help              26180 non-null float64
medical_products          26180 non-null float64
search_and_rescue         26180 non-null float64
security                  26180 non-null float64
military                  26180 non-null float64
child_alone               26180 non-null float64
water                     26180 non-null float64
food                      26180 non-null float64
shelter                   26180 non-null float64
clothing                  26180 non-null float64
money       

In [10]:
y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [11]:
X.isnull().sum()

0

### 2. Write a tokenization function to process your text data

In [19]:

def tokenize(text):
    url_regex ='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex,text)
    for url in detected_urls:
        text = text.replace(url,"urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for token in tokens:
        clean_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_tokens)
    return text
    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [20]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [22]:
X_train.shape

(19635,)

In [ ]:
def evaluate_model(model, X_test, y_test, category_names):
    y_pred = model.predict(X_test)
    class_report = classification_report(y_test, y_pred, target_names=category_names)
    print(class_report)
    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [23]:
pred = pipeline.predict(X_test)

In [24]:
#rint(classification_report(y_test,pred))
category_names=y.columns
for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], pred[:,i], target_names=category_names), '...................................................')


related :
                        precision    recall  f1-score   support

               related       0.33      0.13      0.19      1587
               request       0.76      0.91      0.83      4908
                 offer       0.00      0.00      0.00        50

           avg / total       0.65      0.71      0.67      6545
 ...................................................
request :
                        precision    recall  f1-score   support

               related       0.83      0.98      0.90      5423
               request       0.34      0.05      0.09      1122

           avg / total       0.75      0.82      0.76      6545
 ...................................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6519
               request       0.00      0.00      0.00        26

           avg / total       0.99      1.00      0.99      6545
 ...........................

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd4a6717378>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [26]:
parameters = {
    'clf__estimator__n_estimators':[10],
    'clf__estimator__min_samples_split':[3]
}


cv = GridSearchCV(pipeline,param_grid = parameters,n_jobs=2,verbose=4)

In [27]:
cv.fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.13353705118411, total= 1.7min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.13139801375095492, total= 1.8min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.13277310924369748, total=  55.0s


[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  3.0min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  3.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__min_samples_split': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=4)

In [28]:

cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.13257, std: 0.00089, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 10}]

In [29]:
print(cv.best_params_)

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 10}


In [30]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
y_pred = optimised_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]), '...................................................')

related :
             precision    recall  f1-score   support

          0       0.30      0.11      0.16      1587
          1       0.76      0.92      0.83      4908
          2       0.00      0.00      0.00        50

avg / total       0.64      0.72      0.66      6545
 ...................................................
request :
             precision    recall  f1-score   support

          0       0.84      0.97      0.90      5423
          1       0.37      0.07      0.12      1122

avg / total       0.76      0.82      0.77      6545
 ...................................................
offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545
 ...................................................
aid_related :
             precision    recall  f1-score   support

          0       0.60      0.76      0.67      38

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [33]:
with open('MLClassifier.pkl','wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [34]:
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')


def load_data(database_filepath):
    engine = create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql_table('DisasterResponse.db',engine)
    X = df.message
    y = (df[df.columns[4:]]).astype(int)
    category_names = y.columns
    return X, y, category_names 
    

def tokenize(text):
    url_regex ='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex,text)
    for url in detected_urls:
        text = text.replace(url,"urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens=[]
    for token in tokens:
        clean_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


def build_model():
    pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
    ])
       # parameters set to this due to reduce the size of pkl file, which were too large (600MB) for uploading to github with my previous parameters.
    parameters = {
    'clf__estimator__n_estimators':[10],
    'clf__estimator__min_samples_split':[2]
    }
    model = GridSearchCV(pipeline, param_grid=parameters, n_jobs=2, verbose=4)
    return model
    

def evaluate_model(model, X_test, y_test, category_names):
    y_pred = model.predict(X_test)
    class_report = classification_report(y_test, y_pred, target_names=category_names)
    print(class_report)
    


def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)
def main():
    #if len(sys.argv) == 3:
    model_filepath = "classifier.pkl"
    database_filepath ="DisasterResponse.db"
    print('Loading data...\n    DATABASE: {}'.format(database_filepath))
    X, y, category_names = load_data(database_filepath)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    print('Building model...')
    model = build_model()

    print('Training model...')
    model.fit(X_train, y_train)

    print('Evaluating model...')
    evaluate_model(model, X_test, y_test, category_names)

    print('Saving model...\n    MODEL: {}'.format(model_filepath))
    save_model(model, model_filepath)

    print('Trained model saved!')

    #else:
        #print('Please provide the filepath of the disaster messages database '\
              #'as the first argument and the filepath of the pickle file to '\
              #'save the model to as the second argument. \n\nExample: python '\
              #'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Loading data...
    DATABASE: DisasterResponse.db
Building model...
Training model...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.15539959896877686, total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.1527001862197393, total= 2.2min
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.1587165162584157, total=  58.9s


[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  3.8min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  3.8min finished


Evaluating model...


ValueError: Unknown label type: (       related  request  offer  aid_related  medical_help  medical_products  \
22784        1        0      0            1             0                 0   
25621        1        0      0            0             0                 0   
25391        1        0      0            1             0                 0   
4201         1        0      0            0             0                 0   
9753         1        0      0            1             0                 0   
12563        1        1      0            1             0                 0   
23533        1        0      0            0             0                 0   
20699        1        0      0            0             0                 0   
23068        1        0      0            1             0                 0   
9863         1        0      0            0             0                 0   
9929         1        0      0            0             0                 0   
24450        1     